In [ ]:
###  INTRODUCTION: DSA 210 TERM PROJECT - CINEMATIC ECONOMY DATA PROCESSING & EDA

# --------------------------
# STEP 1: SETUP AND DATA LOADING
# --------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
from scipy import stats
import warnings
warnings.filterwarnings('ignore') # Suppress minor warnings for cleaner output

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

try:
    movies_path = 'raw_data/tmbd_5000_movies.csv' # Your main movie file name
    cpi_path = 'raw_data/US CPI.csv'
    unemp_path = 'raw_data/UNRATE.csv'

    df_movies = pd.read_csv(movies_path)
    df_cpi = pd.read_csv(cpi_path)
    df_unemp = pd.read_csv(unemp_path)

    print("STATUS: All datasets loaded successfully.")

except FileNotFoundError as e:
    print(f"FATAL ERROR: One or more data files not found. Check your path: {e}")
    raise # Stop execution

# --------------------------
# STEP 2: DATA CLEANING AND FILTERING
# --------------------------

DATE_COL = 'release_date'
BUDGET_COL = 'budget'
REVENUE_COL = 'revenue'

df_movies[DATE_COL] = pd.to_datetime(df_movies[DATE_COL], errors='coerce')
df_movies['Year'] = df_movies[DATE_COL].dt.year

# Filter for the relevant study period and valid financial data
df_movies = df_movies[(df_movies['Year'] >= 2010) & (df_movies['Year'] <= 2024)].copy()
df_movies = df_movies[(df_movies[BUDGET_COL] > 1000) & (df_movies[REVENUE_COL] > 1000)].copy()

print(f"STATUS: Cleaning completed. Filtered Dataset Size: {df_movies.shape}")

# --------------------------
# STEP 3: FEATURE ENGINEERING (DATA ENRICHMENT)
# --------------------------

# A. INFLATION ADJUSTMENT (Real_Budget)
# Assumes df_cpi has 'Year' and 'CPI_Avg' columns
df_movies = pd.merge(df_movies, df_cpi[['Year', 'CPI_Avg']], on='Year', how='left')
cpi_base_year = df_movies['CPI_Avg'].max() # Uses the latest CPI value as the base for adjustment

df_movies['Real_Budget'] = df_movies[BUDGET_COL] * (cpi_base_year / df_movies['CPI_Avg'])
df_movies['Real_Revenue'] = df_movies[REVENUE_COL] * (cpi_base_year / df_movies['CPI_Avg'])
print("STATUS: Inflation Adjustment (Real_Budget) completed.")

# B. UNEMPLOYMENT RATE MERGE (Economic Context)
# Assumes df_unemp has 'Year' and 'Unemployment_Rate' columns
df_movies = pd.merge(df_movies, df_unemp[['Year', 'Unemployment_Rate']], on='Year', how='left')
print("STATUS: Unemployment Rate merged.")

# C. HOLIDAY FLAG (Is_Holiday Flag)
# This uses the holidays library to check for major US holidays.
us_holidays = holidays.US()

def is_holiday_release(date):
    if pd.isnull(date): return 0
    for i in range(-7, 8): # Check 7 days before and after release date
        check_date = date + pd.Timedelta(days=i)
        if check_date in us_holidays:
            return 1
    return 0

df_movies['Is_Holiday'] = df_movies[DATE_COL].apply(is_holiday_release)
print("STATUS: Holiday Flag (Is_Holiday) created.")

# --------------------------
# STEP 4: EXPLORATORY DATA ANALYSIS (EDA)
# --------------------------

df_final = df_movies.copy()

# 1. Relationship Visualization (Budget vs Revenue)
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Real_Budget', y='Real_Revenue', hue='Is_Holiday', data=df_final, alpha=0.6)
plt.title('EDA: Real Budget vs. Real Revenue (Colored by Holiday)')
plt.xlabel('Real Budget (Inflation Adjusted)')
plt.ylabel('Real Revenue')
plt.xlim(0, df_final['Real_Budget'].quantile(0.99))
plt.ylim(0, df_final['Real_Revenue'].quantile(0.99))
plt.show()

# 2. Correlation Matrix (Heatmap)
corr_cols = ['Real_Budget', 'Real_Revenue', 'Unemployment_Rate', 'Is_Holiday', 'vote_average']
plt.figure(figsize=(10, 8))
sns.heatmap(df_final[corr_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('EDA: Correlation Matrix of Key Features')
plt.show()

# --------------------------
# STEP 5: INITIAL HYPOTHESIS TESTING (Nov 28 Deadline)
# --------------------------

# Testing H3: Films released during major US Holiday windows achieve statistically higher revenue.
# We use a T-Test to compare the means of the two groups.

holiday_revenue = df_final[df_final['Is_Holiday'] == 1]['Real_Revenue'].dropna()
non_holiday_revenue = df_final[df_final['Is_Holiday'] == 0]['Real_Revenue'].dropna()

t_stat, p_value = stats.ttest_ind(holiday_revenue, non_holiday_revenue, equal_var=False)

print("\n--- HYPOTHESIS TEST RESULTS (H3 - Holiday Effect) ---")
print(f"Average Revenue (Holiday): ${holiday_revenue.mean():,.2f}")
print(f"Average Revenue (Non-Holiday): ${non_holiday_revenue.mean():,.2f}")
print(f"P-Value: {p_value:.4f}")

if p_value < 0.05:
    print("CONCLUSION: P-value < 0.05. We REJECT the Null Hypothesis (H0) for H3.")
    print("There is a statistically significant difference in revenue.")
else:
    print("CONCLUSION: P-value is NOT significant. We FAIL TO REJECT the Null Hypothesis (H0) for H3.")